In [3]:
import numpy as np
import torch
import torch.nn as nn

from lamp import *

In [4]:
datax = [
    [[0.1517, 0.5443, 0.1664, 0.2968, 0.1047, 0.2303, 0.7809, 0.8742]],
    [[0.6482, 0.8415, 0.3437, 0.5621, 0.8318, 0.5978, 0.2179, 0.5826]],
    # [[0.0959, 0.5683, 0.8281, 0.2220, 0.7972, 0.3602, 0.9269, 0.3220]],
    # [[0.2017, 0.4027, 0.9019, 0.3093, 0.2855, 0.0226, 0.3115, 0.5141]],
]

# data shape
batch, chan_in, length = np.array(datax).shape

# conv shape
k_size = 3
chan_out = 2

params = np.random.rand(k_size, chan_in, chan_out)
np_input = np.array(datax).reshape(batch, length, chan_in)
th_input = torch.Tensor(datax)

In [5]:
conv = Conv1D(k_size, chan_in, chan_out)
output = conv.forward(np_input)
print(output.shape)
np.round_(output.reshape(batch, chan_out, -1), decimals=4)

(2, 6, 2)


array([[[-0.0338,  0.0385, -0.036 , -0.0084, -0.0223,  0.0184],
        [-0.0214, -0.0054, -0.0281, -0.0055, -0.0598,  0.0363]],

       [[-0.0716,  0.0461, -0.0613, -0.0085, -0.0541,  0.0156],
        [-0.0717,  0.0401, -0.0665,  0.0238, -0.0455, -0.0137]]])

In [6]:
conv_ = nn.Conv1d(chan_in, chan_out, k_size)
output = conv_(th_input)
output.shape

torch.Size([2, 2, 6])

In [7]:
maxp = MaxPool1D(k_size, stride=1)
# output = maxp.forward(np_input)
# output.reshape(batch, chan_in, -1)

In [8]:
maxp_ = nn.MaxPool1d(k_size, stride=1)
output = maxp_(th_input)
output

tensor([[[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742]],

        [[0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]]])

In [9]:
stride = 3
d_out = (length - k_size) // stride + 1
res = np.zeros((batch, d_out, chan_in))

for k in range(d_out):
    t1, t2 = stride * k, 2 * (k_size // 2) + k * stride + 1
    res[:, k, :] = np.amax(np_input[:, t1:t2, :], axis=1)

res.reshape(batch, chan_in, -1)

array([[[0.5443, 0.2968]],

       [[0.8415, 0.8318]]])

In [10]:
res = np.zeros((batch, d_out, chan_in))
print(np_input)

for k in range(d_out):
    t1, t2 = stride * k, 2 * (k_size // 2) + k * stride + 1
    amax = np.amax(np_input[:, t1:t2, :], axis=1).flatten()
    # print(amax)
    # print(np.where(np_input == amax, 1, 0))
    idx = np.where(np.all(np_input == amax, axis=-1))[0]
    print(idx)
    # res[:, k, :] = np_input[:, idx:]

# res.reshape(batch, chan_in, -1)

[[[0.1517]
  [0.5443]
  [0.1664]
  [0.2968]
  [0.1047]
  [0.2303]
  [0.7809]
  [0.8742]]

 [[0.6482]
  [0.8415]
  [0.3437]
  [0.5621]
  [0.8318]
  [0.5978]
  [0.2179]
  [0.5826]]]
[]
[]


In [11]:
input = maxp_(th_input)

m6 = nn.Flatten()
output = m6(input)
output.size(), output

(torch.Size([2, 6]),
 tensor([[0.5443, 0.5443, 0.2968, 0.2968, 0.7809, 0.8742],
         [0.8415, 0.8415, 0.8318, 0.8318, 0.8318, 0.5978]]))

In [12]:
input = maxp.forward(np_input)

m5 = Flatten()
output = m5.forward(input)
output.shape, output

TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
a = np.ones((4, 3, 2))
print(a)
a = np.insert(a,[0],0 ,axis= 1)

print(a)
print(a.shape)

for i in range(0, 8):
    if i%2 != 0 :   
        a = np.insert(a,i,0 ,axis= 1)
print(a.shape)        


[[[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]]
[[[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[0. 0.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]]
(4, 4, 2)
(4, 8, 2)


In [23]:
stride = 2
m1 = Conv1D(k_size, chan_in, chan_out, stride=stride)
delta = m1.forward(np_input)



print(delta.shape)
d_out = length

res = np.zeros((batch, length, chan_in))

if stride>1:
    for i in range(0,m1.sizeOut +1):
        if i%stride != 1 :
            delta = np.insert(delta, i, 0, axis= 1)
print(delta.shape)
for kd, kr in enumerate(list(range(0, d_out, stride))):
    print("kd", kd, " kr", kr)
    t1, t2 = 0, k_size
    d1 = -kd - 2 + k_size
    d2 = -kd - 2

    if kd == 0:
        t1, t2 = 0, kd + 1
        d1, d2 = None, -1

    elif kd < k_size - 1:
        print("1")
        t1, t2 = 0, kd + 1
        d1, d2 = -1, -kd - 2

    elif kd > d_out - (k_size + 1):
        print("2")
        t1, t2 = k_size - d_out + kd, k_size
        d1, d2 = -kd + 2, 0

    print(d1, d2, kr, kd)
    print(np.flip(params, axis=1)[np.newaxis, t1:t2, :, :])
    print(delta[:, d2:d1, np.newaxis, :])

    res[:, kr, :] = np.sum(
        np.flip(params, axis=1)[np.newaxis, t1:t2, :, :]
        * delta[:, d2:d1, np.newaxis, :],
        axis=(1, 3),
    )

res.shape, np_input.shape


(2, 3, 2)
(2, 5, 2)
kd 0  kr 0
None -1 0 0
[[[[0.0905757  0.43566882]]]]
[[[[ 0.03607543 -0.03728876]]]


 [[[ 0.10302157  0.00078448]]]]
kd 1  kr 2
1
-1 -3 2 1
[[[[0.0905757  0.43566882]]

  [[0.50732829 0.46798477]]]]
[[[[ 0.          0.        ]]

  [[ 0.03017006 -0.01237305]]]


 [[[ 0.          0.        ]]

  [[ 0.07290172 -0.04585366]]]]
kd 2  kr 4
-1 -4 4 2
[[[[0.0905757  0.43566882]]

  [[0.50732829 0.46798477]]

  [[0.06005251 0.89085364]]]]
[[[[ 0.04177868 -0.02974892]]

  [[ 0.          0.        ]]

  [[ 0.03017006 -0.01237305]]]


 [[[ 0.10148305 -0.02762399]]

  [[ 0.          0.        ]]

  [[ 0.07290172 -0.04585366]]]]
kd 3  kr 6
-2 -5 6 3
[[[[0.0905757  0.43566882]]

  [[0.50732829 0.46798477]]

  [[0.06005251 0.89085364]]]]
[[[[ 0.          0.        ]]

  [[ 0.04177868 -0.02974892]]

  [[ 0.          0.        ]]]


 [[[ 0.          0.        ]]

  [[ 0.10148305 -0.02762399]]

  [[ 0.          0.        ]]]]


((2, 8, 1), (2, 8, 1))